In [ ]:
! pip install evaluate

In [ ]:
! pip install -U accelerate

In [ ]:
! pip install -U transformers

In [1]:
import accelerate
import transformers

transformers.__version__, accelerate.__version__

/Users/shahriyar/miniforge3/envs/llm10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


('4.47.1', '1.2.1')

In [2]:
from datasets import DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

In [3]:
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import numpy as np
from transformers import DataCollatorWithPadding

In [4]:
dataset_dict = load_dataset("shawhin/phishing-site-classification")

In [5]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 2100
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 450
    })
})

In [6]:

# Load model directly
model_path = "google-bert/bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_path)

id2label = {0: "Safe", 1: "Not Safe"}
label2id = {"Safe": 0, "Not Safe": 1}
model = AutoModelForSequenceClassification.from_pretrained(model_path, 
                                                           num_labels=2, 
                                                           id2label=id2label, 
                                                           label2id=label2id,)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight True
bert.embeddings.position_embeddings.weight True
bert.embeddings.token_type_embeddings.weight True
bert.embeddings.LayerNorm.weight True
bert.embeddings.LayerNorm.bias True
bert.encoder.layer.0.attention.self.query.weight True
bert.encoder.layer.0.attention.self.query.bias True
bert.encoder.layer.0.attention.self.key.weight True
bert.encoder.layer.0.attention.self.key.bias True
bert.encoder.layer.0.attention.self.value.weight True
bert.encoder.layer.0.attention.self.value.bias True
bert.encoder.layer.0.attention.output.dense.weight True
bert.encoder.layer.0.attention.output.dense.bias True
bert.encoder.layer.0.attention.output.LayerNorm.weight True
bert.encoder.layer.0.attention.output.LayerNorm.bias True
bert.encoder.layer.0.intermediate.dense.weight True
bert.encoder.layer.0.intermediate.dense.bias True
bert.encoder.layer.0.output.dense.weight True
bert.encoder.layer.0.output.dense.bias True
bert.encoder.layer.0.output.LayerNorm.weight True


In [9]:

# freeze base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [10]:
# print layers
for name, param in model.named_parameters():
   print(name, param.requires_grad)

bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.output.dense.bias False
bert.encoder.layer.0.output.Lay

In [11]:
# define text preprocessing
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [12]:
# tokenize all datasetse
tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map: 100%|██████████| 450/450 [00:00<00:00, 30872.76 examples/s]


In [13]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [15]:
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred
    
    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs, references=labels)['roc_auc'],3)
    
    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes, references=labels)['accuracy'],3)
    
    return {"Accuracy": acc, "AUC": auc}

In [16]:
# hyperparameters
lr = 2e-4
batch_size = 8
num_epochs = 10

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/var/folders/mx/8hc3c2316511q43kfytnj1kw0000gq/T/ipykernel_16164/2732273287.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
 10%|█         | 263/2630 [00:35<07:02,  5.60it/s]

{'loss': 0.4942, 'grad_norm': 1.696757435798645, 'learning_rate': 0.00018, 'epoch': 1.0}


                                                  
 10%|█         | 263/2630 [00:40<07:02,  5.60it/s]

{'eval_loss': 0.4239687919616699, 'eval_Accuracy': 0.782, 'eval_AUC': 0.911, 'eval_runtime': 4.907, 'eval_samples_per_second': 91.707, 'eval_steps_per_second': 11.616, 'epoch': 1.0}


 20%|██        | 526/2630 [00:54<02:51, 12.29it/s]

...

                                                   
100%|██████████| 2630/2630 [03:13<00:00, 15.55it/s]

{'eval_loss': 0.29816487431526184, 'eval_Accuracy': 0.871, 'eval_AUC': 0.951, 'eval_runtime': 3.0968, 'eval_samples_per_second': 145.313, 'eval_steps_per_second': 18.406, 'epoch': 10.0}


100%|██████████| 2630/2630 [03:13<00:00, 13.57it/s]

{'train_runtime': 193.7822, 'train_samples_per_second': 108.369, 'train_steps_per_second': 13.572, 'train_loss': 0.35932009156665873, 'epoch': 10.0}


TrainOutput(global_step=2630, training_loss=0.35932009156665873, metrics={'train_runtime': 193.7822, 'train_samples_per_second': 108.369, 'train_steps_per_second': 13.572, 'total_flos': 706603239165360.0, 'train_loss': 0.35932009156665873, 'epoch': 10.0})

In [18]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)

100%|██████████| 57/57 [00:03<00:00, 16.22it/s]

{'Accuracy': 0.889, 'AUC': 0.945}


In [21]:
trainer

In [22]:
trainer.save_model("model_1")
tokenizer.save_pretrained("model_pre_1")


('model_pre_1/tokenizer_config.json',
 'model_pre_1/special_tokens_map.json',
 'model_pre_1/vocab.txt',
 'model_pre_1/added_tokens.json',
 'model_pre_1/tokenizer.json')

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# # Load the model and tokenizer
# model = AutoModelForSequenceClassification.from_pretrained("path_to_saved_model")
# tokenizer = AutoTokenizer.from_pretrained("path_to_saved_model")


In [42]:
# Input text for testing
input_text = "youtube.com/watch?v=aEtmx-nW7fY"
# input_text = "paypal-en-cg-bin-webscr-cmd.com.plantillasmlibre.com.ar/paypal/paypal"

# Tokenize the input
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)


In [43]:
import torch
device = torch.device("cpu")
model = model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

In [46]:
# Make predictions
outputs = model(**inputs)
logits = outputs.logits

# Convert logits to probabilities or predicted labels
import torch
predicted_class = torch.argmax(logits, dim=1).item()

print(f"Predicted class: {predicted_class}")
# print(f"Predicted class: {predicted_class}")


Predicted class: 0


In [47]:
import torch
import torch.nn.functional as F

# Convert logits to probabilities
probabilities = F.softmax(logits, dim=1)

# Extract confidence score and predicted class
confidence, predicted_class = torch.max(probabilities, dim=1)

print(f"Predicted class: {predicted_class.item()}")
print(f"Confidence score: {confidence.item()}")


Predicted class: 0
Confidence score: 0.9884912967681885


In [48]:
# Print all class probabilities
for i, prob in enumerate(probabilities[0]):
    print(f"Class {i}: {prob.item() * 100:.2f}%")


Class 0: 98.85%
Class 1: 1.15%


In [41]:
from sklearn.metrics import classification_report, accuracy_score

# Example true labels and predictions
true_labels = [1, 0, 2]  # Replace with your true labels
predicted_labels = [1, 0, 1]  # Replace with your predictions

# Compute and display metrics
print(classification_report(true_labels, predicted_labels))
print(f"Accuracy: {accuracy_score(true_labels, predicted_labels):.2f}")


              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       0.50      1.00      0.67         1
           2       0.00      0.00      0.00         1

    accuracy                           0.67         3
   macro avg       0.50      0.67      0.56         3
weighted avg       0.50      0.67      0.56         3

Accuracy: 0.67


/Users/shahriyar/miniforge3/envs/llm10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/shahriyar/miniforge3/envs/llm10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/shahriyar/miniforge3/envs/llm10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr